In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_final = df_test

In [ ]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])
for col in df.select_dtypes(include='object').columns:
    df_test[col] = le.fit_transform(df_test[col])

In [ ]:
df.isnull().sum()

,0
Id,0
MSSubClass,0
MSZoning,0
LotFrontage,259
LotArea,0
...,...
MoSold,0
YrSold,0
SaleType,0
SaleCondition,0


In [ ]:
df.isnull().sum().sum()

np.int64(348)

In [ ]:
df.isnull().sum()[df.isnull().sum() > 0]

,0
LotFrontage,259
MasVnrArea,8
GarageYrBlt,81


In [ ]:
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())
df_test['LotFrontage'] = df_test['LotFrontage'].fillna(df_test['LotFrontage'].median())
df['LotFrontage'].fillna(df.groupby('Neighborhood')['LotFrontage'].transform('median'), inplace=True)
df_test['LotFrontage'].fillna(df_test.groupby('Neighborhood')['LotFrontage'].transform('median'), inplace=True)

drop_cols = ['Utilities', '...', 'MasVnrArea', 'GarageYrBlt']
df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')
df_test = df_test.drop(columns=[col for col in drop_cols if col in df_test.columns], errors='ignore')

<ipython-input-103-aef10d42c9d7>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['LotFrontage'].fillna(df.groupby('Neighborhood')['LotFrontage'].transform('median'), inplace=True)
<ipython-input-103-aef10d42c9d7>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

In [ ]:
df.shape

(1460, 78)

In [ ]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [ ]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

In [ ]:
none_cols = ['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
             'BsmtFinType2', 'FireplaceQu', 'GarageFinish',
             'GarageQual', 'GarageCond']

def preprocess(df_test):
  for col in none_cols:
      df_test[col] = df_test[col].fillna('None')

  mode_cols = ['MSZoning', 'Exterior1st', 'Exterior2nd', 'KitchenQual', 'Functional', 'SaleType', 'GarageCars', 'GarageArea', 'PoolQC', 'Fence', 'MiscFeature', 'BsmtHalfBath', 'BsmtFullBath', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2', "Alley"]
  for col in mode_cols:
      df_test[col] = df_test[col].fillna(df_test[col].mode()[0])
  return df_test

df = preprocess(df)
df_test = preprocess(df)


In [ ]:
df_test.isnull().sum()[df_test.isnull().sum() > 0]

,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.isnull().sum().sum())
print(X_test.isnull().sum().sum())

0
0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC

In [ ]:
from sklearn.ensemble import RandomForestRegressor

my_classifier = RandomForestRegressor(n_estimators=1000, random_state=42)
my_classifier.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
preds = my_classifier.predict(X_test)
r2 = r2_score(y_test, preds)
print(f"Accuracy: {r2:.2f}")

Accuracy: 0.89


In [ ]:
print(len(df_test['Id']))
print(len(preds))

1460
292


In [ ]:
df_test = df_test.drop('SalePrice', axis=1)

KeyError: "['SalePrice'] not found in axis"

In [ ]:
prediction = my_classifier.predict(df_test)
prediction = prediction[:-1]

In [ ]:
output = pd.DataFrame({
    'Id': df_final['Id'],
    'SalePrice': prediction
})

In [ ]:
output.to_csv('SalePricePrediction5.csv', index=False)

In [ ]:
from google.colab import files
files.download('SalePricePrediction5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>